daily_run estimates strikes and number of contracts for a put selling strategy on equity indices such as S&P 500. 

Algorithm is drawn from  JUREK, J. W. and STAFFORD, E. (2015), The Cost of Capital for Alternative Investments. The Journal of Finance, 70: 2185-2226. doi:10.1111/jofi.12269- 

Requires an open session of either IB Gateway or TWS from Interactive Brokers

In [1]:
from option_daily_prod import OptionMarket, SpxOptionAsset, RSL2OptionAsset
import numpy as np
import pandas as pd
import nest_asyncio
from time import time

before = time()
nest_asyncio.apply()

spx_trades = OptionMarket(SpxOptionAsset())
rsl_trades = OptionMarket(RSL2OptionAsset())

In [13]:
z_score = np.array([-1, -1.5, -2])
leverage = np.array(range(1, 4, 1))
num_expiries = 3
illiquid_equity = 500000
after = time()
spx = True
if spx:
    trd_choice = spx_trades.form_trade_choice(z_score, num_expiries, 'P')
else:
    trd_choice = rsl_trades.form_trade_choice(z_score, num_expiries, 'P')
    
last_price = trd_choice.spot
after_2 = time()
print('elapsed:', np.round(after - before, 1))
print('elapsed:', np.round(after_2 - before, 1))
print(last_price)
print(trd_choice.sigma)
print(trd_choice.account_value)

elapsed: 102.4
elapsed: 129.4
2657.74
19.78
[AccountValue(account='U2463043', tag='NetLiquidationByCurrency', value='2771571.8811', currency='BASE', modelCode='')]


In [3]:
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction:row;
}
"""
HTML('<style>{}</style>'.format(CSS))


In [4]:
_=[display(df) for df in [trd_choice.strike_grid(), trd_choice.premium_grid(), trd_choice.prices_grid()]]

,-1.0,-1.5,-2.0
2018.12.21,2530,2470,2430
2019.01.18,2470,2380,2320
2019.02.15,2405,2295,2215


,-1.0,-1.5,-2.0
2018.12.21,14.60,8.00,5.35
2019.01.18,21.35,12.25,8.45
2019.02.15,26.00,15.35,10.50


,-1.0,-1.5,-2.0
2018.12.21,14.4/14.8,7.9/8.1,5.2/5.5
2019.01.18,21.1/21.6,12.1/12.4,8.3/8.6
2019.02.15,25.7/26.3,15.1/15.6,10.3/10.7


In [5]:

prct_decline = trd_choice.pct_otm_grid(last_price)
format_dict = {x: '{:.2%}' for x in prct_decline.to_dict()}
display(prct_decline.style.format(format_dict))

,-1.0,-1.5,-2.0
2018.12.21,-4.94%,-7.19%,-8.69%
2019.01.18,-7.19%,-10.57%,-12.83%
2019.02.15,-9.63%,-13.77%,-16.77%


In [6]:
capital_at_risk = float(trd_choice.account_value[0].value) + illiquid_equity
premium_list = []
premium_list =  [(trd_choice.premium_grid() * contractTable.round() * 100) \
                 for contractTable in trd_choice.option_lots(leverage, capital_at_risk)]

_ = [display(df.style.format('${:,.0f}')) for df in premium_list]  

,-1.0,-1.5,-2.0
2018.12.21,"$18,980","$10,400","$7,490"
2019.01.18,"$27,755","$17,150","$11,830"
2019.02.15,"$36,400","$21,490","$15,750"


,-1.0,-1.5,-2.0
2018.12.21,"$37,960","$21,600","$14,445"
2019.01.18,"$57,645","$34,300","$23,660"
2019.02.15,"$72,800","$44,515","$31,500"


,-1.0,-1.5,-2.0
2018.12.21,"$56,940","$32,000","$21,935"
2019.01.18,"$85,400","$51,450","$36,335"
2019.02.15,"$106,600","$66,005","$47,250"


In [7]:
premium_list_pct = []
premium_list_pct = [(trd_choice.premium_grid() * lev) / last_price for lev in leverage]
_ = [display(df.style.format('{:.2%}')) for df in premium_list_pct]  

,-1.0,-1.5,-2.0
2018.12.21,0.55%,0.30%,0.20%
2019.01.18,0.80%,0.46%,0.32%
2019.02.15,0.98%,0.58%,0.39%


,-1.0,-1.5,-2.0
2018.12.21,1.10%,0.60%,0.40%
2019.01.18,1.60%,0.92%,0.64%
2019.02.15,1.95%,1.15%,0.79%


,-1.0,-1.5,-2.0
2018.12.21,1.65%,0.90%,0.60%
2019.01.18,2.41%,1.38%,0.95%
2019.02.15,2.93%,1.73%,1.18%


In [8]:

_ = [display(contractTable.round().style.format('{:.0f}')) for \
     contractTable in trd_choice.option_lots(leverage, capital_at_risk)]
    

,-1.0,-1.5,-2.0
Lev 1,,,
2018.12.21,13,13,14
2019.01.18,13,14,14
2019.02.15,14,14,15


,-1.0,-1.5,-2.0
Lev 2,,,
2018.12.21,26,27,27
2019.01.18,27,28,28
2019.02.15,28,29,30


,-1.0,-1.5,-2.0
Lev 3,,,
2018.12.21,39,40,41
2019.01.18,40,42,43
2019.02.15,41,43,45


In [9]:
margin_dlr = [trd_choice.margin(last_price) * df for df in trd_choice.option_lots(leverage, capital_at_risk)]
_ = [display(df.style.format('${:,.0f}')) for df in margin_dlr]  

,-1.0,-1.5,-2.0
2018.12.21,"$540,145","$453,565","$428,744"
2019.01.18,"$470,920","$368,404","$336,630"
2019.02.15,"$422,654","$342,790","$348,000"


,-1.0,-1.5,-2.0
2018.12.21,"$1,080,290","$942,019","$826,864"
2019.01.18,"$978,064","$736,809","$673,260"
2019.02.15,"$845,309","$710,065","$696,000"


,-1.0,-1.5,-2.0
2018.12.21,"$1,620,434","$1,395,584","$1,255,609"
2019.01.18,"$1,448,984","$1,105,213","$1,033,935"
2019.02.15,"$1,237,774","$1,052,855","$1,044,000"


In [10]:
marge = trd_choice.margin(trd_choice.spot)
lots = trd_choice.option_lots([2], capital_at_risk)
margin = lots[0] * marge
display(margin.style.format('${:,.0f}'))

,-1.0,-1.5,-2.0
Lev 2,,,
2018.12.21,"$1,080,290","$942,019","$826,864"
2019.01.18,"$978,064","$736,809","$673,260"
2019.02.15,"$845,309","$710,065","$696,000"


In [11]:
notional_dlr=[df * trd_choice.strike_grid() * 100 for df in trd_choice.option_lots(leverage, capital_at_risk)]
_ = [display(df.style.format('${:,.0f}')) for df in notional_dlr]  

,-1.0,-1.5,-2.0
Lev 1,,,
2018.12.21,"$3,289,000","$3,211,000","$3,402,000"
2019.01.18,"$3,211,000","$3,332,000","$3,248,000"
2019.02.15,"$3,367,000","$3,213,000","$3,322,500"


,-1.0,-1.5,-2.0
Lev 2,,,
2018.12.21,"$6,578,000","$6,669,000","$6,561,000"
2019.01.18,"$6,669,000","$6,664,000","$6,496,000"
2019.02.15,"$6,734,000","$6,655,500","$6,645,000"


,-1.0,-1.5,-2.0
Lev 3,,,
2018.12.21,"$9,867,000","$9,880,000","$9,963,000"
2019.01.18,"$9,880,000","$9,996,000","$9,976,000"
2019.02.15,"$9,860,500","$9,868,500","$9,967,500"


In [12]:
capital_at_risk

3271571.8811